In [2]:
import pandas as pd
import numpy as np

## Tablas de ratings

In [3]:
df1 = pd.read_csv('1.csv')
df2 = pd.read_csv('2.csv')
df3 = pd.read_csv('3.csv')
df4 = pd.read_csv('4.csv')
df5 = pd.read_csv('5.csv')
df6 = pd.read_csv('6.csv')
df7 = pd.read_csv('7.csv')
df8 = pd.read_csv('8.csv')

##### número de filas por data frame y total que debería tener una vez concatenado

In [4]:
num_filas = []

for i in range(1, 9):
    df_name = f"df{i}"
    df = locals()[df_name]  # get the data frame by name
    print(f"{df_name} shape: {df.shape}")
    x = df.shape
    num_filas.append(x[0])
sum_total_filas = sum(num_filas)
print('Número total de filas:', sum_total_filas)

df1 shape: (1500000, 4)
df2 shape: (1500000, 4)
df3 shape: (1500000, 4)
df4 shape: (1500000, 4)
df5 shape: (1500000, 4)
df6 shape: (1500000, 4)
df7 shape: (524289, 4)
df8 shape: (1500000, 4)
Número total de filas: 11024289


##### Número de valores faltantes por data frame

In [5]:
df_isna = []
for d in (df1, df2, df3, df4, df5, df6, df7, df8):
    df_isna.append(d.isna().sum().sum())
df_isna

[0, 0, 0, 0, 0, 0, 0, 0]

##### Antes de concatenar los df's de ratings verificamos:
##### que tengan nombres de columnas iguales

In [6]:
columnas = []
nombre_igual = []
for d in (df1, df2, df3, df4, df5, df6, df7, df8):
    columnas.append(list(d.columns))
for c in range(7):
    nombre_igual.append(columnas[c] == columnas[c+1])
nombre_igual

[True, True, True, True, True, True, True]

##### que tengan el mismo dtype en cada columna

In [7]:
columnas1 = list(df1.columns)
x = {}
y = {}
n = 0
for d in (df1, df2, df3, df4, df5, df6, df7, df8):
    n+=1
    m = str(n)
    x[m]=[]
    y[m] = 'df' + m
    for col in columnas1:
        x[m].append(d[col].dtype)
        
dataframes_dtypes = pd.DataFrame.from_dict(x, orient='index')
dataframes_dtypes.columns = columnas1
dataframes_dtypes.rename(index=y, inplace=True)
dataframes_dtypes

,userId,rating,timestamp,movieId
df1,int64,float64,int64,object
df2,int64,float64,int64,object
df3,int64,float64,int64,object
df4,int64,float64,int64,object
df5,int64,float64,int64,object
df6,int64,float64,int64,object
df7,int64,float64,int64,object
df8,int64,float64,int64,object


##### Concatenamos df's en reviews y revisamos que el nuevo df tenga número de filas correctas

In [8]:
reviews = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8])
reviews_shape = reviews.shape
if sum_total_filas == reviews_shape[0]:
    print('Número total de filas:', sum_total_filas)
else: 
    print('El número total de filas no concuerda con la suma de las filas de cada dfs')

Número total de filas: 11024289


In [9]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11024289 entries, 0 to 1499999
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   rating     float64
 2   timestamp  int64  
 3   movieId    object 
dtypes: float64(1), int64(2), object(1)
memory usage: 420.5+ MB


#### Revisando el campo 'movieId':
Primero usamos la función extract() de Pandas y una expresión regular para extraer solo las letras de cada elemento de la columna "movieId". La expresión regular [A-Za-z]+ coincide con una o más letras mayúsculas o minúsculas consecutivas en cada cadena. El parámetro expand=False indica que queremos devolver solo una columna en lugar de un DataFrame con una columna por cada grupo de la expresión regular.
Luego, usamos el método unique() en la columna de letras para obtener los valores únicos. Estos valores únicos son las letras de los elementos en la columna "movieId", ignorando los números.

In [10]:
import re

letras = reviews["movieId"].str.extract(r'([A-Za-z]+)', expand=False)
unicos_letras = letras.unique()
unicos_letras

array(['as', 'ns', 'hs', 'ds'], dtype=object)

##### Buscamos filas duplicadas

In [11]:
"""
duplicados = reviews.duplicated(['userId', 'rating', 'timestamp', 'movieId'])
num_duplicados = sum(duplicados)
print('Número de filas duplicadas en data frame "reviews":', num_duplicados)
"""

'\nduplicados = reviews.duplicated([\'userId\', \'rating\', \'timestamp\', \'movieId\'])\nnum_duplicados = sum(duplicados)\nprint(\'Número de filas duplicadas en data frame "reviews":\', num_duplicados)\n'

##### Quitamos filas duplicadas

In [12]:
"""
reviews = reviews.drop_duplicates()
reviews_new_shape = reviews.shape
if (sum_total_filas - num_duplicados) == (reviews_new_shape[0]):
    print('Operación exitosa')
elif (sum_total_filas - num_duplicados) < (reviews_new_shape[0]):
    print('Se quitaron filas de menos')
else:
    print('Se quitaron fila de más')
"""

"\nreviews = reviews.drop_duplicates()\nreviews_new_shape = reviews.shape\nif (sum_total_filas - num_duplicados) == (reviews_new_shape[0]):\n    print('Operación exitosa')\nelif (sum_total_filas - num_duplicados) < (reviews_new_shape[0]):\n    print('Se quitaron filas de menos')\nelse:\n    print('Se quitaron fila de más')\n"

#### Convirtiendo campo timestamp a datetime y formato AAAA-mm-dd

In [13]:
# apply strip() to all columns
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
df['timestamp'] = df['timestamp'].dt.strftime('%Y-%m-%d')

#### Agrupando por movieId con su media de rating

In [14]:
reviews = df.groupby('movieId')['rating'].mean()

#### Exportando a csv 

In [15]:
#reviews.to_csv('reviews_mean.csv', index=True)

## Tablas de plataformas

#### Cargamos archivos csv a data frames

In [16]:
amazon = pd.read_csv('amazon_prime_titles.csv')
netflix = pd.read_csv('netflix_titles.csv')
hulu = pd.read_csv('hulu_titles.csv')
disney = pd.read_csv('disney_plus_titles.csv')

#### Valores faltantes por tabla de plataforma

In [17]:
for plat in (amazon, netflix, hulu, disney):
    print(plat.isna().sum().sum())

22161
4307
8627
888


#### Nombres de las columnas de los data frames de las plataformas

In [18]:
if (amazon.columns.equals(disney.columns) and
    amazon.columns.equals(hulu.columns) and
    amazon.columns.equals(netflix.columns)):
    print("Todos los data frames tienes los mismos nombres de columnas")
else:
    print("No tienen todos los mismos nombres de columnas")

Todos los data frames tienes los mismos nombres de columnas


#### Data Types por columnas en los data frames 

In [19]:
columnas1 = list(amazon.columns)
x = {}
y = {'1':'amazon', '2':'disney', '3':'hulu', '4':'neflix'}
n = 0
for d in (amazon, disney, hulu, netflix):
    n+=1
    m = str(n)
    x[m]=[]
   
    for col in columnas1:
        x[m].append(d[col].dtype)
        
plataforms_dtypes = pd.DataFrame.from_dict(x, orient='index')
plataforms_dtypes.columns = columnas1
plataforms_dtypes.rename(index=y, inplace=True)
plataforms_dtypes

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
amazon,object,object,object,object,object,object,object,int64,object,object,object,object
disney,object,object,object,object,object,object,object,int64,object,object,object,object
hulu,object,object,object,object,float64,object,object,int64,object,object,object,object
neflix,object,object,object,object,object,object,object,int64,object,object,object,object


##### Cambiando hulu['cast'] a dtype object

In [20]:
hulu['cast'] = hulu['cast'].astype('object')

#### Datos faltantes por columna y data frame

### Hacemos compatible el show_id de las tablas de plataformas con el de las tablas de ratings
###### Generar campo id: Cada id se compondrá de la primera letra del nombre de la plataforma, seguido del show_id ya presente en los datasets (ejemplo para títulos de Amazon = as123)

In [21]:
disney['show_id'] = disney['show_id'].str.replace('s', 'ds')
amazon['show_id'] = amazon['show_id'].str.replace('s', 'as')
hulu['show_id'] = hulu['show_id'].str.replace('s', 'hs')
netflix['show_id'] = netflix['show_id'].str.replace('s', 'ns')

### Concatenamos los data frames de las plataformas en data frame 'plata'

#### pd.concat() tiene por default el parámetro join=outer. 
El outer join mantendrá todas las filas de cada uno de los data frames y llenará los campos sin valores con NaN.

In [22]:
plata = pd.concat([amazon, disney, hulu, netflix])

##### Cambiamos el nombre de la columna 'rating' (que causa confusión, porque en realidad se trata de la clasificación de la película) por 'classification'

In [23]:
plata.rename(columns={'rating': 'classification'}, inplace=True)

### Reemplazondo Nan en 'classification'('rating')
###### Los valores nulos del campo rating deberán reemplazarse por el string “G” (corresponde al maturity rating: “general for all audiences”

In [24]:
# Reemplazamos los valores NaN con 'G'
plata['classification'] = plata['classification'].fillna('G')

In [25]:
classification_unicas = plata['classification'].unique()

##### La columna 'classification' (antes 'rating') contiene en algunas filas valores de la columna 'duration' por lo que estos se extraen y se reemplazan por 'G'. Si en la misma fila se encuentra valor en la columna 'duration' estos últimos se extraen e insertan en una nueva columna 'duration_type' y el campo 'duration' se insertan los valores extraídos de 'classification'.

In [26]:
plata = plata.reset_index(drop=True)
# create columns for extracted data
plata['temp_classification'] = ''
plata['temp_duration'] = ''

# extract data from classification column
plata.loc[plata['classification'].str.contains('min|Season'), 'temp_classification'] = plata['classification']
plata.loc[plata['classification'].str.contains('min|Season'), 'classification'] = 'NOT RATED'

# extract duration data from corresponding rows
plata.loc[plata['temp_classification'] != '', 'temp_duration'] = plata['duration']
plata.loc[(plata['temp_classification'] != '') & (plata['duration'] != ''), 'duration'] = plata['temp_classification']

# drop temp_classification column if all values are empty
if plata['temp_classification'].isnull().all():
    plata.drop('temp_classification', axis=1, inplace=True)


In [27]:
plata.drop(['temp_classification', 'temp_duration'], axis=1, inplace=True)

#### Formato de fechas
##### De haber fechas, deberán tener el formato AAAA-mm-dd

In [28]:
plata['date_added'] = pd.to_datetime(plata['date_added'].str.strip(), format='%B %d, %Y')
plata['date_added'] = plata['date_added'].dt.strftime('%Y-%m-%d')

### Conversión del campo 'duration' a dos columnas: 'duration_int', 'duration_type'

In [29]:
# Separamos los números y las letras en columnas diferentes
plata[['duration_int', 'duration_type']] = plata['duration'].str.extract(r'(\d+)\s*(\D+)', expand=True)

# Insertamos las nuevas columnas justo después de la columna "duration"
duration_idx = plata.columns.get_loc('duration')
plata['dur_int'] = ''
plata['dur_type'] = ''
plata.insert(duration_idx + 1, 'duration_int', plata.pop('duration_int'))
plata.insert(duration_idx + 2, 'duration_type', plata.pop('duration_type'))
plata.drop(['duration'], axis=1, inplace=True)

In [30]:
plata.drop(['dur_int', 'dur_type'], axis=1, inplace=True)

#### Los campos de texto deberán estar en minúsculas, sin excepciones

In [31]:
plata[['type', 'title', 'director', 'cast', 'country', 'classification','duration_type','listed_in', 'description']] = plata[['type', 'title', 'director', 'cast', 'country', 'classification','duration_type','listed_in', 'description']].apply(lambda x: x.str.lower())

In [32]:
# apply strip() to all columns
plata = plata.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [33]:
plata['duration_type'] = plata['duration_type'].str.replace('seasons', 'season')

In [34]:
#plata.to_csv('plata.csv', index=False)

#### Merge para crear columna con los scores de las peliculas en la tabla de plataformas

In [35]:
users = pd.read_csv('reviews_mean.csv')

# Merge the two dataframes on their common column
merged_df = pd.merge(plata, users, left_on='show_id', right_on='movieId')

# Create a new column 'scores' in 'plata' dataframe by copying 'rating' column values from 'users'
plata['scores'] = merged_df['rating']

# Save the updated dataframe to a csv file
plata.to_csv('platas.csv', index=False)